In [41]:
import numpy as np 
import tensorflow as tf 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
import spacy
from tensorflow.keras import layers 
#import unidecode
#from word2number import w2n
#import pycontractions
#import contractions 


In [20]:
data = pd.read_csv(r'D:\project\resnet_basics\all_tickets.csv')
#data['ticket_type'].plot(kind = 'hist')

In [21]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48549 entries, 0 to 48548
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             47837 non-null  object
 1   body              48549 non-null  object
 2   ticket_type       48549 non-null  int64 
 3   category          48549 non-null  int64 
 4   sub_category1     48549 non-null  int64 
 5   sub_category2     48549 non-null  int64 
 6   business_service  48549 non-null  int64 
 7   urgency           48549 non-null  int64 
 8   impact            48549 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 3.3+ MB
None


# EXTRACTING REQUIRED VALUES

In [25]:
df = data.loc[:, ['title','body','category','sub_category1','sub_category1','urgency',]].copy()
df.head()

,title,body,category,sub_category1,sub_category1,urgency
0,NaN,hi since recruiter lead permission approve req...,4,2,2,3
1,connection with icon,icon dear please setup icon per icon engineers...,6,22,22,3
2,work experience user,work experience user hi work experience studen...,5,13,13,3
3,requesting for meeting,requesting meeting hi please help follow equip...,5,13,13,3
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,4,2,2,3


In [26]:
size = len(df)
size

48549

# ADDING EXTRA COLUMN

In [27]:
# 
df['department'] = np.random.randint(0,4,size=size)

In [28]:
df.isna().sum()

title            712
body               0
category           0
sub_category1      0
sub_category1      0
urgency            0
department         0
dtype: int64

In [29]:
# taking care of null values
df['title'] = df['title'].fillna("Value Missing")

In [42]:
def strip_html_tags(text): 
    """remove html tags""" 
    soup = BeautifulSoup(text, "html.parser") 
    stripped_text = soup.get_text(separator=" ") 
    return stripped_text 

def word2number(text):
    help_dict = { 
    'one': '1', 
    'two': '2', 
    'three': '3', 
    'four': '4', 
    'five': '5', 
    'six': '6', 
    'seven': '7', 
    'eight': '8', 
    'nine': '9', 
    'zero' : '0'
} 

    return  " ".join([help_dict[word] for word in text.split() if word in help_dict])

import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

# applying lemmatization
df['title'] = df['title'].apply(lemmatize_text)
df['body'] = df['body'].apply(lemmatize_text)



In [43]:
 from sklearn.feature_extraction.text import TfidfVectorizer 
 from sklearn.feature_extraction import text 
 mystopwords = text.ENGLISH_STOP_WORDS
 vectorizer_body = TfidfVectorizer(
     strip_accents='ascii',
     lowercase=True,
     stop_words='english',
 )
 
 body = vectorizer_body.fit_transform(df['body']) 


 vectorizer_title = TfidfVectorizer(
     strip_accents='ascii',
     lowercase=True,
     stop_words=mystopwords
 )
 
 title = vectorizer_title.fit_transform(df['title'])




In [50]:
shape

NameError: name 'shape' is not defined

In [44]:
# extracting tags
tags = df.loc[:,['category','sub_category1','sub_category1']]
tags

,category,sub_category1,sub_category1,sub_category1,sub_category1
0,4,2,2,2,2
1,6,22,22,22,22
2,5,13,13,13,13
3,5,13,13,13,13
4,4,2,2,2,2
...,...,...,...,...,...
48544,4,21,21,21,21
48545,4,9,9,9,9
48546,5,32,32,32,32
48547,4,21,21,21,21


In [45]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
tags = enc.fit_transform(tags)


In [59]:
priority = tf.keras.utils.to_categorical(df['urgency'])
department = tf.keras.utils.to_categorical(df['department'])

In [56]:
title = tf.keras.Input(shape=(size,), name = 'title')
text_body = tf.keras.Input(shape=(size,), name = 'body') 
tags = tf.keras.Input(shape=(size,), name = 'tags')

features = layers.Concatenate()([title, text_body, tags])
batch_normalization = layers.BatchNormalization()(features)

features = layers.Dense(512, activation = 'relu')(batch_normalization)
batch_normalization = layers.BatchNormalization()(features)
dropout = layers.Dropout(0.3)(batch_normalization)

features = layers.Dense(512, activation = 'relu')(dropout)
batch_normalization = layers.BatchNormalization()(features)
dropout = layers.Dropout(0.3)(batch_normalization)

features = layers.Dense(512, activation = 'relu')(dropout)
batch_normalization = layers.BatchNormalization()(features)
dropout = layers.Dropout(0.3)(batch_normalization)
 

